### This notebook is to generate df of Positions, that is resulted in our strategy

In [1]:
import pandas as pd
from math import sqrt
import math

import requests
import json
import numpy as np

import UNI_v3_funcs as liq_amounts
import load_data

### Query Functions

In [68]:
def get_cprice_at_mint(token0, token1, feeTier, mint_time, decimal0, decimal1):
    
    # get token_ids
    token0_id = load_data.get_token_id(token0)
    token1_id = load_data.get_token_id(token1)
    pool_id = load_data.get_pool_id(token0_id, token1_id, feeTier)
        
    # query price at that specific time
    query_ = """
    {{
      poolHourDatas(first:1,
      where:{{ pool: "{}",
      periodStartUnix: {} }}
      ){{
        tick
      }}
    }}""".format(pool_id, mint_time)
    print(query_)
    
    query_result_ = load_data.run_query(query_)
    json_data_ = json.loads(query_result_.text)
    
    ctick = int(json_data_['data']['poolHourDatas'][0]['tick'])
    cprice = tick_to_price(ctick,decimal0, decimal1)
    
    return cprice

def get_decimals(token_symbol):
    query_ = """
    {{
      tokens(first:10,
      where:{{ symbol: "{}"}}
      ){{
        decimals
      }}
    }}""".format(token_symbol)
    
    query_result_ = load_data.run_query(query_)
    json_data_ = json.loads(query_result_.text)
    decimal_ = float(json_data_['data']['tokens'][0]['decimals'])
    
    return decimal_

### Calculations Functions

In [55]:
def price_to_tick(price_, decimal0_, decimal1_):
    # convert price (token1 by token0) to tick
#     tick_ = math.floor(math.log(sqrt(price_), math.sqrt(1.0001)))
    tick_ = math.floor( math.log(price_*10**(decimal1_-decimal0_), 1.0001) )
    
    return tick_

def tick_to_price(tick_,decimal0_, decimal1_):
    price_ = (1.0001**tick_/10**(decimal1_-decimal0_))
#     price_ = 1.0001**tick_
    return price_

In [19]:
# L_you calculation / inRange
# 4B. JNP: Use 'get_liquidity' function to calculate liquidity as a function of amounts and price range
def get_liquidity0(sqrtA,sqrtB,amount0,decimals):
    
    if (sqrtA > sqrtB):
          (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
    liquidity=amount0/((2**96*(sqrtB-sqrtA)/sqrtB/sqrtA)/10**decimals)
    return liquidity

def get_liquidity1(sqrtA,sqrtB,amount1,decimals):
    
    if (sqrtA > sqrtB):
        (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
    liquidity=amount1/((sqrtB-sqrtA)/2**96/10**decimals)
    return liquidity

def get_liquidity(tick,tickA,tickB,amount0,amount1,decimal0,decimal1):
    
        sqrt=(1.0001**(tick/2)*(2**96))
        sqrtA=(1.0001**(tickA/2)*(2**96))
        sqrtB=(1.0001**(tickB/2)*(2**96))
        if (sqrtA > sqrtB):
            (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
        if sqrt<=sqrtA:
            
            liquidity0=get_liquidity0(sqrtA,sqrtB,amount0,decimal0)
            inRange = False
            return liquidity0, inRange
        elif sqrt<sqrtB and sqrt>sqrtA:

            liquidity0=get_liquidity0(sqrt,sqrtB,amount0,decimal0)
            liquidity1=get_liquidity1(sqrtA,sqrt,amount1,decimal1)
            
            liquidity=liquidity0 if liquidity0<liquidity1 else liquidity1
            inRange = True
            return liquidity, inRange
        
        else:
            liquidity1=get_liquidity1(sqrtA,sqrtB,amount1,decimal1)
            inRange = False
            return liquidity1, inRange

In [27]:
# Calculate Fees 
def get_fees(lower_tick_, upper_tick_, ctick_,  L_Pool_, swap_Volume_, amt0_, amt1_, decimal0_, decimal1_, feeTier):
    # (lower_tick_, upper_tick_) from df_positions
    # (ctick, L_Pool, swap_volume) from merge
    # (amt0, amt1) - this changes based on swaps, but doesnt matter, bacause liquidity always constant
    # (decimal0, decimal1, feetier) - fixed
    
#     tick_ = price_to_tick(cprice_)
#     tickA_ = price_to_tick(lower_)
#     tickB_ = price_to_tick(upper_)
    L_you_, inRange_ = get_liquidity(ctick_, lower_tick_,upper_tick_, amt0_, amt1_, decimal0_, decimal1_)

    if inRange_:
        pool_share_ = (L_you_/L_Pool_)
        pool_fee_rate_ = float(feeTier) / 10000
        fees = pool_share_ * swap_Volume_ * pool_fee_rate_ / 100
    else:
        fees = 0
        
    return fees

In [6]:
# Step 4: use JNP's code
def get_impermanent_loss():
    return IL

In [7]:
# Step 4
def get_PNL():
    return pnl

In [8]:
# Step 5
def get_cost():
    return cost

### Positions Genrations

In [56]:
def get_positions_fixed_strategy(mint_time, burn_time, upper_, lower_, decimal0_, decimal1_):
    
    df_positions = pd.DataFrame()
    df_positions['periodStartUnix'] = list(range(mint_time, burn_time+1, 3600))
    df_positions['upper'] = upper_
    df_positions['lower'] = lower_
    df_positions['upper_tick'] = price_to_tick(upper_, decimal0_, decimal1_)
    df_positions['lower_tick'] = price_to_tick(lower_, decimal0_, decimal1_)
    
    return df_positions

### PNL Wrapper code

In [10]:
def calculate_pnl_from_positions(df_merged, df_positions):

    merge_feat = ['periodStartUnix', 'tick','liquidity','amountUSD']
    df_positions_fees = df_positions.merge(df_merged[merge_feat], how='left', on='periodStartUnix')
    df_positions_fees['liquidity'] = df_positions_fees['liquidity'].astype(float)

    df_positions_fees = df_positions_fees.reset_index() # cleaning it just in case
    for index, row in df_positions_fees.iterrows():
        row = row.copy()
        fees = get_fees(row['lower_tick'],row['upper_tick'],row['tick'],float(row['liquidity']),row['amountUSD'],
                                                MINT_AMT0, MINT_AMT1, DECIMAL0, DECIMAL1, FEETIER)
        df_positions_fees.loc[index, 'fees'] = fees
        
    return df_positions_fees

### Main

In [11]:
### input: money USD, upper and lower
# maths:
# p(i) = 1.0001**i , where i = tick
# i = math.floor(math.log(sqrt(P)/(2**96), math.sqrt(1.0001)))

# of positions:
# liquidity L = sqrt(token0_amt*token1__amt) ?? > may need the complicated 3 cases maths
# - Fee income = (L_you/L_pool) * swap volume under fixed time period (USD) * pool_fee_rate/100

In [57]:
# STEP 0:  PROVIDE BY USERS
UPPER = 1/1844.6164  # token1 per token0 (in terms of price)
LOWER = 1/2858.3641  # 1/2376.6075 
FEETIER = '3000' # 0.3 = 3000/10000
TOKEN0 = 'USDC'
TOKEN1 = 'WETH'
MINT_TIME = 1622390400 # needs to start with the hour (unixTime % 3600 = 0) 
BURN_TIME = 1627578000

In [72]:
# Get required constants
DECIMAL0 = get_decimals(TOKEN0)
DECIMAL1 = get_decimals(TOKEN1)
print(DECIMAL0, DECIMAL1)

6.0 18.0


In [71]:
CPRICE = get_cprice_at_mint(TOKEN0, TOKEN1, FEETIER, MINT_TIME, DECIMAL0, DECIMAL1)
print(CPRICE, 1/CPRICE)

 
get_token_id: USDC
{'data': {'tokens': [{'id': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'name': 'USD Coin', 'symbol': 'USDC'}]}}
 
get_token_id: WETH
{'data': {'tokens': [{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'name': 'Wrapped Ether', 'symbol': 'WETH'}]}}

 get_pool_id for feeTier: 3000
{'data': {'pools': [{'feeTier': '3000', 'id': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8', 'token0': {'symbol': 'USDC'}, 'token1': {'symbol': 'WETH'}}]}}

    {
      poolHourDatas(first:1,
      where:{ pool: "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",
      periodStartUnix: 1622390400 }
      ){
        tick
      }
    }
0.0004165877310455854 2400.4547553287744


In [59]:
# STEP 1 (TODO): CALCULATED BY ALGORITHM, JNP's amounts_relation ?
# THIS CHANGES over time when swaps happens, and we need this to calculate IL
MINT_AMT0 = 252208.7053
MINT_AMT1 = 99.0604

In [41]:
# 1.0001**198486

In [62]:
# STEP 3 (TODO): Generate df_positions
df_positions = get_positions_fixed_strategy(MINT_TIME, BURN_TIME, UPPER, LOWER, DECIMAL0, DECIMAL1)

In [64]:
# STEP 4 (TODO): Calculate Fees, IL and PNL at each hour, and append to df_positions
df_merged = pd.read_csv('../data/df_merged_tmp_USDC-WETH-3000-timestamp-1627574400-1620169200.csv')
df_positions_fees = calculate_pnl_from_positions(df_merged, df_positions)
df_positions_fees['fees'].sum()



48579.45029866004

In [22]:
# STEP 5 (TODO): also accoutning for minting/burning/swaping costs into PNL

In [67]:
df_positions_fees.head(10)

,index,periodStartUnix,upper,lower,upper_tick,lower_tick,tick,liquidity,amountUSD,fees
0,0,1622390400,0.000542,0.00035,201120,196740,198486.0,2.955344e+19,2.303598e+07,97.555589
1,1,1622394000,0.000542,0.00035,201120,196740,198382.0,3.093428e+19,9.405126e+06,36.509524
2,2,1622397600,0.000542,0.00035,201120,196740,198254.0,3.040995e+19,1.087038e+07,40.873872
3,3,1622401200,0.000542,0.00035,201120,196740,198290.0,2.947882e+19,7.864582e+06,30.922322
4,4,1622404800,0.000542,0.00035,201120,196740,198305.0,4.947310e+19,1.400576e+07,33.000362
5,5,1622408400,0.000542,0.00035,201120,196740,198355.0,5.023388e+19,1.489900e+07,35.243610
6,6,1622412000,0.000542,0.00035,201120,196740,198426.0,3.123902e+19,7.951657e+06,31.100406
7,7,1622415600,0.000542,0.00035,201120,196740,198521.0,3.075176e+19,7.257897e+06,29.963479
8,8,1622419200,0.000542,0.00035,201120,196740,198650.0,2.638387e+19,2.960617e+07,150.395405
9,9,1622422800,0.000542,0.00035,201120,196740,198871.0,2.434524e+19,1.916945e+07,111.276283


In [ ]:
df_merged = pd.read_csv('../data/df_merged_DAI-WETH-3000-timestamp-1627466400-1620158400.csv')
watch_list = ['periodStartUnix',  
              'txCount', 'swaps_txCount', # check data integrity
              'amount0', 'amount1', 'amountUSD', # swaps data, note: amountUSD is sqrt(P)
              'tick', 'liquidity', 'sqrtPrice', 'tvlUSD', # pool data at that hour (liquidity is for the whole pool)
              'pool.token0.symbol', 'pool.token1.symbol', # token data
              'token0Price', 'token1Price'
             ]
# https://github.com/Uniswap/uniswap-v3-subgraph/blob/main/schema.graphql
df_merged[watch_list].sample(15)